In [5]:
!pip install dune-client tf2onnx ezkl onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 34.4 MB/s eta 0:00:00


In [ ]:
# from dune_client.client import DuneClient
# dune = DuneClient("API_KEY_DUNE")
# query_result = dune.get_latest_result(6648623)

In [ ]:
# import pandas as pd
# import time
# df = pd.DataFrame(query_result.result.rows)

# # Cek apakah ada data yang kosong (None) seperti priority_fee_gwei di Arbitrum
# # Kita isi dengan 0 agar tidak merusak training AI nanti
# df.fillna(0, inplace=True)

# df.to_csv("beetrap_dataset.csv", index=False)
# print(f"🎉 Berhasil! {len(df)} baris data disimpan sebagai 'beetrap_dataset.csv'")
# print(df.head())

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import tf2onnx
import ezkl
import json

# --- 1. DATA PROCESSING (6 FEATURES) ---
df = pd.read_csv('beetrap_dataset_update.csv') #
features = ['gas_price_gwei', 'priority_fee_gwei', 'gas_usage_ratio', 'gas_used', 'native_value', 'tx_index']
X = df[features].values.astype(np.float32)
y = df['label'].apply(lambda x: 1.0 if x == 'bot' else 0.0).values.astype(np.float32)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) #
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# SIMPAN UNTUK RUST (Wajib!)
print(f"Mean: {scaler.mean_.tolist()}")
print(f"Scale: {scaler.scale_.tolist()}")

# --- 2. TENSORFLOW MODEL ---
model = tf.keras.Sequential([
    layers.Input(shape=(6,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1) # Output Logit
])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, verbose=0)



Mean: [0.9686767258720472, 0.75661699955974, 0.5725549928540364, 570146.967, 0.4807524878626883, 54.8847]
Scale: [7.238926964973418, 7.100118667856041, 0.18967953026975654, 871444.9154728459, 11.583387250982357, 85.2014871109067]


In [7]:
y_pred_logits = model.predict(X_test)
y_pred = (y_pred_logits > 0.0).astype(int) # Convert logits to binary predictions

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion Matrix:
[[827 199]
 [202 772]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.81      0.80      1026
         1.0       0.80      0.79      0.79       974

    accuracy                           0.80      2000
   macro avg       0.80      0.80      0.80      2000
weighted avg       0.80      0.80      0.80      2000



In [13]:
# --- 3. EXPORT & EZKL ---
# Menggunakan Opset 12 agar stabil di EZKL

# Define a concrete function to wrap the model's call method
# Explicitly define input signature with a batch size of 1 to match EZKL input later
@tf.function(input_signature=[tf.TensorSpec(shape=(1, 6), dtype=tf.float32, name="input")])
def serve_model(inputs):
    return model(inputs)

# Convert the tf.function to ONNX using from_function
# Pass the input_signature from the serve_model function directly
model_proto, _ = tf2onnx.convert.from_function(
    serve_model,
    input_signature=serve_model.input_signature,
    opset=12
)

with open("network.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())


ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7cc8bd46e020>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


In [15]:
# --- 4. KONVERSI KE ONNX MENGGUNAKAN TF2ONNX ---
import onnx
model_path = "network.onnx"

spec = (tf.TensorSpec((1, 6), tf.float32, name="input"),)

# Manually set output_names to prevent AttributeError in tf2onnx
# This is a workaround for some tf2onnx versions/environments
model.output_names = ['output']

# tf2onnx mengubah model Keras langsung ke file ONNX
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=12)
onnx.save(model_proto, model_path)
print(f"✅ Model ONNX berhasil dibuat: {model_path}")

ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x7cc8bd46e020>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


✅ Model ONNX berhasil dibuat: network.onnx


In [16]:
# --- 4. CALIBRATION & VERIFIER ---
# Memastikan skala matematika ZK akurat untuk gas price yang kecil
data_path = "input.json"
with open(data_path, "w") as f:
    json.dump(dict(input_data=[X_test[0].tolist()]), f)

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "public"
run_args.output_visibility = "public"



In [17]:
# This creates a settings.json file with default parameters based on your ONNX model.
ezkl.gen_settings("network.onnx", "settings.json", py_run_args=run_args)


True

In [23]:
# This optimizes the settings for resource usage. You can also use target="accuracy" to optimize for accuracy.
ezkl.calibrate_settings(data_path, "network.onnx", "settings.json", target="accuracy")



ERROR:ezkl.graph.model:[tensor] decomposition error: integer -13384557203 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer -53546256916 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer -214190727515 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer -107115194400 is too large to be represented by base 16384 and n 2
ERROR:ezkl.execute:forward pass failed: "failed to forward: [halo2] General synthesis error"
ERROR:ezkl.graph.model:[tensor] decomposition error: integer 386867944532 is too large to be represented by base 16384

True

In [24]:
# This converts your ONNX model into an optimized format for zero-knowledge proofs, creating the network.ezkl file.

ezkl.compile_circuit("network.onnx", "network.ezkl", "settings.json")

True

In [39]:
await ezkl.get_srs("settings.json", srs_path="kzg.srs")

True

In [44]:
# This creates a witness file from your input data and compiled model.
# Tentukan jalur file
data_path = "input.json"          # File input (biasanya hasil normalisasi)
compiled_model_path = "network.ezkl" # Hasil dari compile_circuit
witness_path = "witness.json"     # File output yang akan dibuat
srs_path = "kzg.srs"              # File SRS yang sudah diunduh

# Menjalankan gen_witness
# Ini akan melakukan forward pass dan menyimpan hasilnya ke witness.json
ezkl.gen_witness(
    data=data_path,
    model=compiled_model_path,
    output=witness_path
)

{'inputs': [['dafbffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   '98fcffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   '661f000000000000000000000000000000000000000000000000000000000000',
   '7cf0ffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   'adfeffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   '433d000000000000000000000000000000000000000000000000000000000000']],
 'outputs': [['8a9effef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430']],
 'max_lookup_inputs': 0,
 'min_lookup_inputs': 0,
 'max_range_size': 16383}

In [48]:
# Generate prove
proof_path = "model.proof"
pk_path = "pk.key" # Proving Key hasil dari setup

# Membuat bukti (Proof Generation)
ezkl.prove(
    witness_path,
    compiled_model_path,
    pk_path,
    proof_path,
    # "single", # Mode pembuktian
)

{'instances': [['dafbffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   '98fcffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   '661f000000000000000000000000000000000000000000000000000000000000',
   '7cf0ffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   'adfeffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430',
   '433d000000000000000000000000000000000000000000000000000000000000',
   '8a9effef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430']],
 'proof': '0x21e5f92b88622b89567264b267ec31d667fb3a8aa8150ac29dba2655c907e49629e640219e05898fc7850471e2c0b675249deecd16d10314b067891253f43639280e989576c556d15679aff37e72fab268e3a083aa27e1cc034c2094d3063af023a342bfb9b36b755c116612b9cd4b3db662af6173374d985a103cea14c07a2a18c61e8e65beca48b7989d50871f8300afe62cbc82e23073e4a535735cf247fa2d6143eecaf0e8ffbeda3868acbc2f430e04df127739727148b2b0a49b59cd9e12b467d936ac3f96a21ca39ef336434eb796edb72b27a651845103923935b381022d3d83866df977c4fea26fb66

In [49]:
vk_path = "vk.key"
settings_path = "settings.json"
sol_code_path = "Verifier.sol"
abi_path = "Verifier.abi"
srs_path = "kzg.srs"

print("🚀 Membuat Smart Contract Verifier...")


ezkl.create_evm_verifier(
    vk_path=vk_path,
    settings_path=settings_path,
    sol_code_path=sol_code_path,
    abi_path=abi_path,
    srs_path=srs_path,
    reusable=False # Set False agar menjadi kontrak mandiri
)

🚀 Membuat Smart Contract Verifier...


<Future pending cb=[<builtins.PyDoneCallback object at 0x7cc8b2d98bd0>()]>

In [ ]:
ezkl.create_evm_vka